In [13]:
import os
import re
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from bert_score import score
from nltk.translate.meteor_score import meteor_score
import nltk

nltk.download('punkt')

def preprocess_text(text):
    """Usuwa formatowanie Markdown i inne niepotrzebne elementy."""
    text = re.sub(r'\*\*.*?\*\*', '', text)  # Usuwa pogrubienia w stylu Markdown
    text = re.sub(r'\d+\.\s', '', text)  # Usuwa numerowanie list
    return text.strip()

def calculate_bleu(reference_text, candidate_text):
    """Oblicza BLEU score między referencją a wygenerowanym tekstem."""
    reference = [reference_text.split()]  # Tokenizacja referencji
    candidate = candidate_text.split()    # Tokenizacja podsumowania
    return sentence_bleu(reference, candidate)

def calculate_rouge(reference_text, candidate_text):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return scorer.score(reference_text, candidate_text)

def calculate_bertscore(reference_text, candidate_text):
    P, R, F1 = score([candidate_text], [reference_text], lang='en')
    return {'precision': P.mean().item(), 'recall': R.mean().item(), 'f1': F1.mean().item()}

def calculate_meteor(reference_text, candidate_text):
    return meteor_score([reference_text], candidate_text)

def process_files(csv_path):
    file_pairs = pd.read_csv(csv_path)
    if file_pairs.empty:
        print("Plik CSV jest pusty!")
        return pd.DataFrame()  # Zwróć pusty DataFrame

    metrics = []

    for _, row in file_pairs.iterrows():
        summary_path = os.path.normpath(row['summary_path'])
        reference_path = os.path.normpath(row['reference_path'])

        print(f"Sprawdzanie ścieżek: {summary_path}, {reference_path}")
        if not os.path.exists(summary_path):
            print(f"Nie znaleziono pliku podsumowania: {summary_path}")
            continue
        if not os.path.exists(reference_path):
            print(f"Nie znaleziono pliku referencyjnego: {reference_path}")
            continue

        with open(summary_path, "r", encoding="utf-8") as f:
            candidate_text = preprocess_text(f.read().strip())
        with open(reference_path, "r", encoding="utf-8") as f:
            reference_text = preprocess_text(f.read().strip())

        if not candidate_text or not reference_text:
            print(f"Pusty plik: {summary_path} lub {reference_path}")
            continue

        print(f"Przetworzony tekst referencyjny: {reference_text[:100]}")
        print(f"Przetworzony tekst podsumowania: {candidate_text[:100]}")

        # Tokenizacja tekstów
        reference_tokens = reference_text.split()
        candidate_tokens = candidate_text.split()

        try:
            bleu_score = calculate_bleu(reference_text, candidate_text)  # Tokenizacja w funkcji
            rouge_scores = calculate_rouge(reference_text, candidate_text)  # Bez zmian
            bertscore = calculate_bertscore(reference_text, candidate_text)  # Bez zmian
            meteor_score_value = meteor_score([reference_tokens], candidate_tokens)  # Dodanie tokenizacji

            metrics.append({
                "summary_path": summary_path,
                "reference_path": reference_path,
                "BLEU": bleu_score,
                "ROUGE_1": rouge_scores["rouge1"].fmeasure,
                "ROUGE_2": rouge_scores["rouge2"].fmeasure,
                "ROUGE_L": rouge_scores["rougeL"].fmeasure,
                "BERTScore": bertscore['f1'],
                "METEOR": meteor_score_value
            })
        except Exception as e:
            print(f"Błąd podczas obliczania metryk dla {summary_path}: {e}")

    metrics_df = pd.DataFrame(metrics)

    # Jeśli brak wyników, zwróć pusty DataFrame
    if metrics_df.empty:
        print("Brak wyników do zapisania!")
    
    return metrics_df




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\G\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:

# Uruchomienie programu
if __name__ == "__main__":
    csv_path = "voiceapp/output/summaries_folder/summaries_list.csv"
    metrics_df = process_files(csv_path)
    save_metrics_to_csv(metrics_df)

Sprawdzanie ścieżek: voiceapp\output\summaries_folder\mistral_7b_20241208_213239\CJG_01_2023_01_14-summary_1.txt, C:\Users\G\Documents\GitHub\audycje.com.pl\content\audio\CJG_01_2023_01_14.txt
Przetworzony tekst referencyjny: ﻿1
[music]
What's going on, Mr. Mariusz?
What did you hear again?
I checked a few forecasts from the
Przetworzony tekst podsumowania: : The text discusses the theme of knowledge and understanding, particularly in the domains of busine


D:\anaconda3\envs\od_zera_do_ai\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sprawdzanie ścieżek: voiceapp\output\summaries_folder\mistral_7b_20241208_213239\CJG_01_2023_01_14-summary_2.txt, C:\Users\G\Documents\GitHub\audycje.com.pl\content\audio\CJG_01_2023_01_14.txt
Przetworzony tekst referencyjny: ﻿1
[music]
What's going on, Mr. Mariusz?
What did you hear again?
I checked a few forecasts from the
Przetworzony tekst podsumowania: : The main themes of this text revolve around the speaker's personal insights into politics, economi


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sprawdzanie ścieżek: voiceapp\output\summaries_folder\mistral_7b_20241208_213239\CJG_01_2023_01_14-summary_3.txt, C:\Users\G\Documents\GitHub\audycje.com.pl\content\audio\CJG_01_2023_01_14.txt
Przetworzony tekst referencyjny: ﻿1
[music]
What's going on, Mr. Mariusz?
What did you hear again?
I checked a few forecasts from the
Przetworzony tekst podsumowania: Key Themes: The key themes of this text revolve around personal insights from an economist and finan


...

Sprawdzanie ścieżek: voiceapp\output\summaries_folder\mistral_7b_20241208_213239\CJG_03_2023_01_28-summary_1.txt, C:\Users\G\Documents\GitHub\audycje.com.pl\content\audio\CJG_03_2023_01_28.txt
Przetworzony tekst referencyjny: ﻿1
[MUSIC]
What's the deal, Mr. Mariusz?
What did you hear there again?
To be honest, I was inspired
Przetworzony tekst podsumowania: : The main themes of the text revolve around the financial success and impact of large-scale product


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sprawdzanie ścieżek: voiceapp\output\summaries_folder\mistral_7b_20241208_213239\CJG_03_2023_01_28-summary_2.txt, C:\Users\G\Documents\GitHub\audycje.com.pl\content\audio\CJG_03_2023_01_28.txt
Przetworzony tekst referencyjny: ﻿1
[MUSIC]
What's the deal, Mr. Mariusz?
What did you hear there again?
To be honest, I was inspired
Przetworzony tekst podsumowania: : The main themes discussed in the text revolve around the financial performance of TV series and mo


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wyniki zapisane do wyniki_metryk.csv


In [28]:
with open("voiceapp/lista.txt", "r") as f:
    references = f.readlines()
    print(references)

['C:/Users/G/Documents/GitHub/audycje.com.pl/content/audio\\CJG_01_2023_01_14.mp3\n', 'C:/Users/G/Documents/GitHub/audycje.com.pl/content/audio\\CJG_02_2023_01_21.mp3\n', 'C:/Users/G/Documents/GitHub/audycje.com.pl/content/audio\\CJG_03_2023_01_28.mp3\n', 'C:/Users/G/Documents/GitHub/audycje.com.pl/content/audio\\CJG_04_2023_02_04.mp3\n', 'C:/Users/G/Documents/GitHub/audycje.com.pl/content/audio\\CJG_05_2023_02_11.mp3\n', 'C:/Users/G/Documents/GitHub/audycje.com.pl/content/audio\\CJG_06_2023_02_18.mp3\n']
